In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
CLIENT_ID = '1ORSOYLVG2OUFDNUHRBWOL2PIG5NODCT52K55REL3OFDTPVE' # your Foursquare ID
CLIENT_SECRET = 'EI5H30VOVG2U31XEFX4TMBYEHCJWAHLTKVVOIPDU1Z1JC2GX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: 1ORSOYLVG2OUFDNUHRBWOL2PIG5NODCT52K55REL3OFDTPVE


In [25]:
# type your answer here
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'San Francisco, CA', 'Los Angeles, CA', 'Toronto, ON','Vancouver, BC']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d145941735") # Chinese Restaurant PLACE CATEGORY ID
    results[city] = requests.get(url).json()

In [26]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

The Foursquare API Only gives us the nearest 100 venues in the city.

Let's first check out their densities by our eyes

In [27]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Chinese restaurants in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of Chinese restaurants in New York, NY =  346
Showing Top 100
Total number of Chinese restaurants in San Francisco, CA =  313
Showing Top 100
Total number of Chinese restaurants in Los Angeles, CA =  298
Showing Top 100
Total number of Chinese restaurants in Toronto, ON =  189
Showing Top 100
Total number of Chinese restaurants in Vancouver, BC =  185
Showing Top 100


In [28]:
maps[cities[0]]

In [29]:
maps[cities[1]]

In [30]:
maps[cities[2]]

In [31]:
maps[cities[3]]

In [32]:
maps[cities[4]]

We can see that New York and Jersey City are the most dense cities with Chinese restaurants. And better than that, they are just one shore away. 

However, Let's have a concrete measure of this density. 

For this I will use some basic statistics. I will get the mean location of the Chinese restaurants which should be near to most of them if they are really dense or far if not. 

Next I will take the average of the distance of the venues to the mean coordinates. 

In [33]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.02154760387508575
San Francisco, CA
Mean Distance from Mean coordinates
0.03853781759961378
Los Angeles, CA
Mean Distance from Mean coordinates
0.09368039505267714
Toronto, ON
Mean Distance from Mean coordinates
0.08986798274323363
Vancouver, BC
Mean Distance from Mean coordinates
0.03726716669932001


In [34]:
maps[cities[0]]

In [35]:
maps[cities[1]]

In [36]:
maps[cities[2]]

In [37]:
maps[cities[3]]

In [38]:
maps[cities[4]]

### We now see that New York is his best option in the US, whereas Vancouver is his best option in Canada. Our tourist's best interest would be to book a hotel near that mean coordinate to surround himself with the 100 Chinese restaurants there!!

#### Enjoy your chinese foods!